In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
import time
import logging
import watchdog.observers 
import watchdog.events 
import os
import threading

In [ ]:
dataframes = []
processed_files = set()
processing_event = threading.Event()

In [ ]:
class Handler(watchdog.events.PatternMatchingEventHandler):
    def __init__(self):

        # Set the patterns for PatternMatchingEventHandler
        watchdog.events.PatternMatchingEventHandler.__init__(self, patterns=['*.csv'],
                                                             ignore_directories=True, case_sensitive=False)
 
    def on_created(self, event):
        print("Watchdog received created event - % s." % event.src_path)
        filename = os.path.basename(event.src_path).split('/')[-1]
        return filename
        # Event is created, you can process it now
            # Check if file already processed



    def on_modified(self, event):
        filename = self.on_created(event)
        print("Watchdog received modified event - % s." % event.src_path)
        if filename not in processed_files:
            processed_files.add(filename)
            historicalSize = -1
            while (historicalSize != os.path.getsize(filename)):
                historicalSize = os.path.getsize(filename)
                time.sleep(1)
            print("file copy has now finished")
            global Created
            Created = True
            file_data = pd.read_csv(filename)
            file_df = pd.DataFrame(file_data)
            dataframes.append([file_df, filename[:-4]])
            return Created # Only return filename if not processed
        else:
            print(f"Skipping duplicate file: {filename}")
            return None


if __name__=='__main__':
    src_path = r"/home/jasmine/encryption/cryptlibs"

    Created = False
    event_handler = Handler()

    observer = watchdog.observers.Observer()
    observer.schedule(event_handler, path=src_path, recursive=True)
    observer.start()
    try:
        while not Created:
            pass
    except KeyboardInterrupt:
        observer.stop()
    finally:
        observer.stop()
        observer.join()


In [ ]:
print(len(dataframes))

In [ ]:
processed_files.clear()

In [ ]:
length = {" Data length":[]}
length[" Data length"].append(14)
for i in range(250, 450, 10):
    length[" Data length"].append(i)

data_length = pd.DataFrame(length)


In [ ]:
data_length.head()

In [ ]:
N = len(dataframes)
for i, dataframe in enumerate(dataframes):
    dataframes[i][0] = dataframes[i][0].rename({' Run Time': 'Run Time' + str(i), ' CPU cycles': 'CPU cycles'+str(i),  ' Throughput': 'Throughput'+str(i)}, axis=1)
    

In [ ]:
runtimes = data_length
cpu_cycles = data_length
throughputs = data_length

for i, df_set in enumerate(dataframes):
    df = df_set[0]
    runtimes = pd.concat([runtimes, df.iloc[:, 2]], axis=1).reindex(runtimes.index)
    cpu_cycles = pd.concat([cpu_cycles, df.iloc[:, 1]], axis=1).reindex(cpu_cycles.index)
    throughputs = pd.concat([throughputs, df.iloc[:, 3]], axis=1).reindex(throughputs.index)
    

In [ ]:
categorised_dfs = [runtimes, cpu_cycles, throughputs]
for i, df in enumerate(categorised_dfs):
    categorised_dfs[i] = categorised_dfs[i].iloc[1:]
runtimes, cpu_cycles, throughputs = categorised_dfs

In [ ]:
runtimes.head()

In [ ]:
for i, dataframe in enumerate(dataframes):
    plt.plot(runtimes.iloc[:,0],runtimes.iloc[:,i+1], label = str(dataframes[i][1]))
#plt.plot(runtimes.iloc[:,0], runtimes.iloc[:,2], color = 'red', label = "AES-192")
#plt.plot(runtimes.iloc[:,0], runtimes.iloc[:,3], color = 'blue', label = "AES-256")
#plt.plot(runtimes.iloc[:,0], runtimes.iloc[:,4], color = 'purple', label = "WolfSSL AES-256")
#plt.plot(runtimes.iloc[:,0], runtimes.iloc[:,5], color = 'black', label = "WolfSSL AES-192")

plt.xlabel("Data length/bytes")
plt.ylabel("Runtime/ms")
plt.title('RSA 3072 OAEP Runtimes')

plt.legend()

plt.show()

In [ ]:
for i, dataframe in enumerate(dataframes):
    plt.plot(cpu_cycles.iloc[:,0],cpu_cycles.iloc[:,i+1], label = str(dataframes[i][1]))

plt.xlabel("Data length/bytes")
plt.ylabel("CPU Cycles/(MB/s)")
plt.title('RSA 3072 OAEP CPU Cycles')

plt.legend()

plt.show()

In [ ]:
for i, dataframe in enumerate(dataframes):
    plt.plot(throughputs.iloc[:,0],throughputs.iloc[:,i+1], label = str(dataframes[i][1]))

#plt.plot(throughputs.iloc[:,0],throughputs.iloc[:,1], color = 'green', label = "AES-128")
#plt.plot(throughputs.iloc[:,0],throughputs.iloc[:,2], color = 'red', label = "AES-192")
#plt.plot(throughputs.iloc[:,0],throughputs.iloc[:,3], color = 'blue', label = "AES-256")
#plt.plot(throughputs.iloc[:,0], throughputs.iloc[:,4], color = 'purple', label = "WolfSSL AES-256")
#plt.plot(throughputs.iloc[:,0], throughputs.iloc[:,5], color = 'black', label = "WolfSSL AES-192")
plt.xlabel("Data length/bytes")
plt.ylabel("Throughput/(B/s)")
plt.title('RSA 3072 OAEP Throughput')

plt.legend()

plt.show()